In [ ]:
!pip install crewai crewai-tools --quiet

!pip install langchain-openai --quiet

!pip install llama-index --quiet

!pip install llama-index-llms-together --quiet

!pip install together --quiet

In [ ]:
!pip install --upgrade crewai --quiet

In [ ]:
!pip install llamaapi --quiet


In [ ]:
import os
from crewai import Agent, Task, Crew, Process
from langchain_openai import OpenAI
from dotenv import load_dotenv
from llama_index.llms.together import TogetherLLM
import base64
from io import BytesIO
import requests
import json
from llamaapi import LlamaAPI
from google.colab import userdata
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="google/gemma-2-27b-it",
    huggingfacehub_api_token = '',
    task="text-generation",
)

# Define the agents
requirements_agent = Agent(
    role="Requirements Specialist",
    goal="Extract and summarize requirements from the requirements document.",
    backstory="You are an expert in gathering and summarizing requirements for software projects.",
    verbose=True,
    allow_delegation=False,
    llm=llm
)

design_agent = Agent(
    role="Design Specialist",
    goal="Create a high-level design document with visual diagrams based on the requirements provided.",
    backstory="You are skilled in translating requirements into a high-level design for software projects, including creating visual diagrams using AI-powered tools.",
    verbose=True,
    allow_delegation=False,
    llm=llm
)

software_development_agent = Agent(
    role="Software Developer",
    goal="Generate the Code required to build a proof of concept based on the high-level design.",
    backstory="You are responsible for implementing the design into working code of system.",
    verbose=True,
    allow_delegation=False,
    llm=llm
)

code_test_agent = Agent(
    role="Testing Specialist",
    goal="Create and execute test cases to ensure the Code meets the requirements.",
    backstory="You are tasked with testing the code to ensure it fulfills the requirements and works correctly.",
    verbose=True,
    allow_delegation=False,
    llm=llm
)

# Function to run the SDLC workflow
def run_sdlc_workflow(initial_requirements):
    print("Starting SDLC Workflow...")

    # Define the tasks
    requirements_task = Task(
        description=f"Extract and summarize the requirements from the following document: {initial_requirements}",
        expected_output="A concise summary of the key requirements for the CRM system.",
        agent=requirements_agent
    )

    design_task = Task(
      description="Create a high-level design document based on the requirements provided by the Requirements Specialist. Include use case diagram, class diagram, ERD, and user interface designs. Use the provided function to generate visual representations for these diagrams.",
      expected_output="A high-level design document detailing the architecture and components of the CRM system, including visual diagrams.",
      agent=design_agent,
      # context={design_with_diagrams}
    )

    development_task = Task(
        description="Generate the Code for the high-level design provided by the Design Specialist.",
        expected_output="A functional Code implementing the core features of the CRM system Give Code.",
        agent=software_development_agent
    )

    testing_task = Task(
        description="Generate and execute test cases to ensure the codebase meets the requirements and works as expected.",
        expected_output="A comprehensive test report detailing the test cases, results, and any identified issues.",
        agent=code_test_agent
    )

    # Create the crew
    sdlc_crew = Crew(
      agents=[requirements_agent, design_agent, software_development_agent, code_test_agent],
      tasks=[requirements_task, design_task, development_task, testing_task],
      verbose=True,
      process=Process.sequential,
    )

    # Kickoff the crew to execute all tasks
    result = sdlc_crew.kickoff()

    print("SDLC Workflow Completed.")
    return result

# Example usage
if __name__ == "__main__":
    initial_requirements = """
    Develop a simple CRM system with the following features:
    1. User authentication
    2. Customer database (CRUD operations)
    3. Task management for customer interactions
    4. Basic reporting functionality
    """

    final_output = run_sdlc_workflow(initial_requirements)
    print("Final Output:", final_output)

Starting SDLC Workflow...
 [2024-09-15 21:38:12][DEBUG]: == Working Agent: Requirements Specialist
 [2024-09-15 21:38:12][INFO]: == Starting Task: Extract and summarize the requirements from the following document: 
    Develop a simple CRM system with the following features:
    1. User authentication
    2. Customer database (CRUD operations)
    3. Task management for customer interactions
    4. Basic reporting functionality
    


> Entering new CrewAgentExecutor chain...
You are Requirements Specialist. You are an expert in gathering and summarizing requirements for software projects.
Your personal goal is: Extract and summarize requirements from the requirements document.To give my best complete final answer to the task use the exact following format:

Thought: I now can give a great answer
Final Answer: my best complete final answer to the task.
Your final answer must be the great and the most complete as possible, it must be outcome described.

I MUST use these formats, my job